# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

Welcome to JupyROOT 6.14/02


In [2]:
# %load python/drawingTools.py


In [3]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v80'
version_V8 = version
version_V9 = 'v122'
version_V10 = 'v122'

# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0_tps', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0_tps', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_tps', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', version_V9, 'ele-V9')
    ]

samples_ele_V10 = [
    Sample('ele_flat2to100_PU0_tps', 'PU0', version_V10, 'ele-V10'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', version_V10+'t', 'ele-V10')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_tps', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200_tps', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0_tps', 'PU0', version_V8, 'V8'),
    Sample('photon_flat8to150_PU200_tps', 'PU200', version_V8, 'V8')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_tps', 'PU0', version_V9, 'photons-V9'),
    Sample('photon_flat8to150_PU200_tps', 'PU200', version_V9, 'photons-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0_tps', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]



gen_part_selections: 14


In [57]:
# %load python/settings.py

# === samples =====================================================
import pprint
import python.plotters_config as plotters



samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_ele_V9

for smp in samples:
    smp.open_file()


sample = 'ele-V9'

# === TP ==========================================================
tps = ['HMvDR', 
       'HMvDRshapeDrCalib',
       'HMvDREmInt',
      ]


# === Load the Histo Primitives ====================================
histo_primitives = samples[0].build_file_primitive_index()
# samples[0].print_file_primitive_index()

# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.tp == tp].tp_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.tp == tp].gen_sel.unique().tolist()


import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


--- TCHistos
# of plots: 4032
# of primitives: 448
--- ClusterHistos
# of plots: 2240
# of primitives: 448
--- Cluster3DHistos
# of plots: 9408
# of primitives: 448
--- ResoHistos
# of plots: 7616
# of primitives: 448
--- GenParticleHistos
# of plots: 5388
# of primitives: 450
--- ClusterConeHistos
# of plots: 3136
# of primitives: 448
--- TPs: 
['HMvDR', 'HMvDRshapeDrCalib', 'HMvDREmInt']
--- TP selections:
{   'HMvDR': [   'Em',
                 'Pt30',
                 'Pt10',
                 'Pt20',
                 'all',
                 'EmPt30',
                 'EmPt20',
                 'EmPt10'],
    'HMvDREmInt': [   'Em',
                      'EmPt20',
                      'EmPt30',
                      'Pt20',
                      'EmPt10',
                      'all',
                      'Pt10',
                      'Pt30'],
    'HMvDRshapeDrCalib': [   'EmPt10',
                             'EmPt20',
                             'Pt20',
                         

In [5]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())




# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]



In [6]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.tp == tp].tp_sel.unique()
    print histo_primitives[histo_primitives.tp == tp].gen_sel.unique()

TP: HMvDR
['Em' 'Pt30' 'Pt10' 'Pt20' 'all' 'EmPt30' 'EmPt20' 'EmPt10']
['GENEtaBCD' 'GENPt40' 'GENEtaBC' 'GENEtaDEPt15' 'GENEtaDPt15'
 'GENEtaBCDPt15' 'GENPt35' 'GENPt15' 'GENEtaBCPt15' 'GENPt30' 'GEN'
 'GENEtaD' 'GENEtaDE' 'GENPt10to25']
TP: HMvDRshapeDrCalib
['EmPt10' 'EmPt20' 'Pt20' 'Pt30' 'all' 'Em' 'Pt10' 'EmPt30']
['GENEtaBC' 'GENEtaDE' 'GENEtaBCD' 'GENPt10to25' 'GENPt15' 'GENEtaD'
 'GENPt40' 'GENEtaDEPt15' 'GENEtaBCDPt15' 'GENPt30' 'GENPt35' 'GEN'
 'GENEtaDPt15' 'GENEtaBCPt15']
TP: HMvDREmInt
['Em' 'EmPt20' 'EmPt30' 'Pt20' 'EmPt10' 'all' 'Pt10' 'Pt30']
['GENPt15' 'GENEtaBCDPt15' 'GENEtaDEPt15' 'GENPt10to25' 'GENPt40'
 'GENEtaDE' 'GENEtaBC' 'GENEtaD' 'GENPt35' 'GENEtaBCPt15' 'GEN' 'GENPt30'
 'GENEtaDPt15' 'GENEtaBCD']


In [8]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.HistoSetEff)



--- TCHistos
# of plots: 4032
# of primitives: 448
--- ClusterHistos
# of plots: 2240
# of primitives: 448
--- Cluster3DHistos
# of plots: 9408
# of primitives: 448
--- ResoHistos
# of plots: 7616
# of primitives: 448
--- GenParticleHistos
# of plots: 5388
# of primitives: 450
--- ClusterConeHistos
# of plots: 3136
# of primitives: 448
--- TCHistos
# of plots: 4032
# of primitives: 448
--- ClusterHistos
# of plots: 2240
# of primitives: 448
--- Cluster3DHistos
# of plots: 9408
# of primitives: 448
--- ResoHistos
# of plots: 7616
# of primitives: 448
--- GenParticleHistos
# of plots: 5388
# of primitives: 450
--- ClusterConeHistos
# of plots: 3136
# of primitives: 448
CPU times: user 4.46 s, sys: 323 ms, total: 4.78 s
Wall time: 5.15 s


In [9]:
%%time

for smp in hplot.data['sample'].unique():
    print 'Sample: {}'.format(smp)
    for pu in hplot.data[(hplot.data['sample'] == smp)].pu.unique():
        for tp in tps:
#         for tp in ['EG', 'TkEleEL']:
            print 'PU: {}, TP: {}'.format(pu, tp)
            for tp_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                if 'Pt' not in tp_sel:
                    continue
#                 tp_sel_den = 'all'
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print '  {} den -> {}'.format(tp_sel, tp_sel_den) 
                for gen_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp) & (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    if gen_sel == 'nomatch' or 'Pt' in gen_sel:
                        continue
#                     print gen_sel
                    hsetden = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel_den, gen_sel)
                    hset = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel, gen_sel)
                    hset[0][0].computeTurnOn(hsetden[0][0].h_num)


Sample: ele-V9
PU: PU0, TP: HMvDR
  EmPt20 den -> Em
  EmPt30 den -> Em
  Pt10 den -> all
  Pt20 den -> all
  EmPt10 den -> Em
  Pt30 den -> all
PU: PU0, TP: HMvDRshapeDrCalib
  EmPt10 den -> Em
  Pt30 den -> all
  EmPt30 den -> Em
  Pt20 den -> all
  EmPt20 den -> Em
  Pt10 den -> all
PU: PU0, TP: HMvDREmInt
  EmPt10 den -> Em
  EmPt20 den -> Em
  EmPt30 den -> Em
  Pt20 den -> all
  Pt30 den -> all
  Pt10 den -> all
PU: PU200, TP: HMvDR
  EmPt20 den -> Em
  EmPt30 den -> Em
  Pt10 den -> all
  Pt20 den -> all
  EmPt10 den -> Em
  Pt30 den -> all
PU: PU200, TP: HMvDRshapeDrCalib
  EmPt10 den -> Em
  Pt30 den -> all
  EmPt30 den -> Em
  Pt20 den -> all
  EmPt20 den -> Em
  Pt10 den -> all
PU: PU200, TP: HMvDREmInt
  EmPt10 den -> Em
  EmPt20 den -> Em
  EmPt30 den -> Em
  Pt20 den -> all
  Pt30 den -> all
  Pt10 den -> all
CPU times: user 21.9 s, sys: 2.42 s, total: 24.3 s
Wall time: 29.3 s


In [11]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele-V9,PU0,HMvDR,EmPt20,GENPt30,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x125416950>
1,ele-V9,PU0,HMvDREmInt,EmPt10,GENEtaDE,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x12549f368>
2,ele-V9,PU0,HMvDR,Em,GENPt10to25,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x12549fef0>
3,ele-V9,PU0,HMvDRCalib,Em,GENEtaDPt15,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x10e2ad950>
4,ele-V9,PU0,HMvDREmInt,all,GENEtaBCD,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x125424518>
5,ele-V9,PU0,HMvDRshapeDrCalib,EmPt10,GENEtaBC,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x125424560>
6,ele-V9,PU0,HMvDRCalib,Pt10,GENEtaBC,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x1100223b0>
7,ele-V9,PU0,HMvDREmInt,EmPt10,GEN,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x10f8ee950>
8,ele-V9,PU0,HMvDREmInt,Em,GENPt40,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x121035518>
9,ele-V9,PU0,HMvDRCalib,EmPt20,GENEtaBCPt15,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x125424710>


In [12]:
hplot.data.tp.unique()

array(['HMvDR', 'HMvDREmInt', 'HMvDRCalib', 'HMvDRshapeDrCalib'], dtype=object)

In [13]:
hplot.data.gen_sel.unique()

array(['GENPt30', 'GENEtaDE', 'GENPt10to25', 'GENEtaDPt15', 'GENEtaBCD',
       'GENEtaBC', 'GEN', 'GENPt40', 'GENEtaBCPt15', 'GENPt35',
       'GENEtaDEPt15', 'GENEtaD', 'GENPt15', 'GENEtaBCDPt15'], dtype=object)

In [14]:
hplot.data['sample'].unique()

array(['ele-V9'], dtype=object)

## Effinciency vs $\eta$

In [15]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
#             print [his.h_num.h_eta.GetName() for his in hsets]

In [20]:
for tp in tps:
    for tp_sel in ['Pt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU0'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')


No match found for: pu: ['PU0'], tp: HMvDRcylind2p5Calib, tp_sel: Pt20, gen_sel: GENPt30, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [24]:
for tp in tps:
    for tp_sel in ['Pt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')


No match found for: pu: ['PU200'], tp: HMvDRcylind2p5Calib, tp_sel: Pt20, gen_sel: GENPt30, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [21]:
for tp in tps:
    for tp_sel in ['Em']:
        for gen_sel in ['GENPt20']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU0'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')


No match found for: pu: ['PU0'], tp: HMvDRcylind2p5Calib, tp_sel: Em, gen_sel: GENPt20, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [25]:
for tp in tps:
    for tp_sel in ['Em']:
        for gen_sel in ['GENPt20']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')


No match found for: pu: ['PU200'], tp: HMvDRcylind2p5Calib, tp_sel: Em, gen_sel: GENPt20, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [22]:
for tp in tps:
    for tp_sel in ['EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU0'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')
     

No match found for: pu: ['PU0'], tp: HMvDRcylind2p5Calib, tp_sel: EmPt20, gen_sel: GENPt30, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [28]:
for tp in tps:
    for tp_sel in ['EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')
     

No match found for: pu: ['PU200'], tp: HMvDRcylind2p5Calib, tp_sel: EmPt20, gen_sel: GENPt30, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [27]:
print sample

ele-V9


In [28]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['all', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tps, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [17]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', pu, 'HMvDR', tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets_new = []
            labels_new = []
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', pu, 'HMvDR', tp_sel, gen_sel)
            hsets_new.extend(hsets)
            labels_new.append('HM 3D (V8 geom.)')
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['V9'], pu, ['HMvDR', 'HMvDRNC0'], tp_sel, gen_sel)
            hsets_new.extend(hsets)
            labels_new.append('HM 3D (V9 geom.)')
            labels_new.append('HM 3D (V9 geom. + tuned FE th.)')

            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tps, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

## Efficiency vs $p_{T}$

In [32]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU0'], ['HMvDR'], ['all', 'Em'], 'GENEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU200'], ['HMvDR'], ['all', 'Em'], 'GENEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU0'], ['HMvDR'], ['all', 'Em'], 'GENEtaD')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU0', 'PU200'], ['HMvDR'], ['all', 'Em'], 'GENEtaD')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [35]:
for pu in ['PU0', 'PU200']:
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', pu, tp, ['all', 'Em'], gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


No match found for: pu: PU0, tp: HMvDRshapeDr, tp_sel: ['all', 'Em'], gen_sel: GENEtaBC, classtype: python.l1THistos.HistoSetEff


TypeError: 'NoneType' object is not iterable

In [48]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Em']:
        for gen_sel in ['GENEtaBC', 'GENEtaD']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tps, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


## Turn-on curves

In [44]:
for tp in ['HMvDR', 'HMvDRshapeDrCalib', 'HMvDREmInt']:
    for tp_sel in ['EmPt20','EmPt30']:
        for gen_sel in ['GENEtaBC']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [47]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0'], ['HMvDRshapeDrCalib'], 'EmPt30', ['GENEtaBC', 'GENEtaD'], debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['HMvDRshapeDrCalib'], 'EmPt30', ['GENEtaBC', 'GENEtaD'], debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [46]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0'], ['HMvDREmInt'], 'EmPt30', ['GENEtaBC', 'GENEtaD'], debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['HMvDREmInt'], 'EmPt30', ['GENEtaBC', 'GENEtaD'], debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [50]:
for tp_sel in ['EmPt20','EmPt30']:
    for gen_sel in ['GENEtaBC']:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0'], ['HMvDR', 'HMvDR', 'HMvDRshapeDrCalib', 'HMvDREmInt' ], tp_sel, gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [54]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'EmPt30']:
            for gen_sel in ['GENEtaBC', 'GENEtaD']:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', pu, tps, tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

## Scaling

In [55]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(3, 0.8, 1)
tf_yc.SetParLimits(0, 0, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [58]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

import array

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo



for pu in ['PU0', 'PU200']:
    for tp in tps:
        pt_points = [10, 20, 30]
        pt_95 = []
        for pt in pt_points:
            tp_sel = 'EmPt{}'.format(pt)
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, ['GENEtaBC'])
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
#                 draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
            tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
            h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]

            result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
            print '------- {}'.format( text)
            print result.Print()
            pt095 = tf_yc.GetX(0.95)
            pt_95.append(pt095)
            
            #                 points.append((tp, pu, pt, pt095))
            draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
            ROOT.gStyle.SetOptFit(11111)
          
        graph = ROOT.TGraph(len(pt_points), array.array('d', pt_points), array.array('d', pt_95))
        graph.SetMarkerStyle(7)
        graph.SetMarkerColor(2)
        stuff.append(graph)
        
        hplot.data = hplot.data.append({'sample': sample,
                                        'pu': pu,
                                        'tp': tp,
                                        'tp_sel': 'Em',
                                        'gen_sel': gen_sel,
                                        'classtype': 'pippo',
                                        'histo': HWrapper(graph),},
                                        ignore_index=True)

                
                

------- PU0, HM+dR(layer) Cl3d, EGId, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM+dR(layer) Cl3d, EGId, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM+dR(layer) Cl3d, EGId, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM #Delta#rho < 0.015 calib., EGId, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM #Delta#rho < 0.015 calib., EGId, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM #Delta#rho < 0.015 calib., EGId, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM+dR(layer) Cl3d EM Int, EGId, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM+dR(layer) Cl3d EM Int, EGId, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, HM+dR(layer) Cl3d EM Int, EGId, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, HM+dR(layer) Cl3d, EGId, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, HM+dR(layer) Cl3d, EGId, p_{T}^{L1}>=20

In [59]:
hplot.data[hplot.data.classtype == 'pippo']

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
896,ele-V9,PU0,HMvDR,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125d2e650>
897,ele-V9,PU0,HMvDR,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125cc6fd0>
898,ele-V9,PU0,HMvDRshapeDrCalib,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125cff6d0>
899,ele-V9,PU0,HMvDREmInt,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125b10f90>
900,ele-V9,PU200,HMvDR,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125534450>
901,ele-V9,PU200,HMvDRshapeDrCalib,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125435f90>
902,ele-V9,PU200,HMvDREmInt,Em,GENEtaD,pippo,<__main__.HWrapper object at 0x125435d50>


In [60]:
ROOT.gStyle.SetOptFit(11111)

for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Em']:
            for gen_sel in ['GENEtaBC', 'GENEtaBCLBrem']:
                hsets, labels, text = hplot.get_histo('pippo', sample, pu, tp, tp_sel, gen_sel)
                hsets[0].Fit('pol1', 'S')

                draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

No match found for: pu: PU0, tp: HMvDR, tp_sel: Em, gen_sel: GENEtaBC, classtype: pippo


TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
ROOT.gStyle.SetOptFit(11111)

for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Em']:
            hsets, labels, text = hplot.get_histo('pippo', sample, pu, tp, tp_sel, ['GENEtaBC', 'GENEtaBCLBrem'])
            hsets[0].Fit('pol1', 'S')
    
            draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

In [ ]:

for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Em']:
            hsets, labels, text = hplot.get_histo('pippo', 'ele-V9', pu, tp, tp_sel, ['GENEtaBC'])
            hsets[0].Fit('pol1', 'S')
    
            draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []


for pu in ['PU0', 'PU200']:
    for tp in ['HMvDRfixed']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:

import numpy as np
from array import array

def f_cb(x, par):
    return ROOT.Math.crystalball_function(x[0], par[0], par[1], par[2], par[3])

tf_cb = ROOT.TF1('cb', f_cb, 0, 100, 4)
tf_cb.SetParameters(10., 4., 3, 20.)

def f_cbi(x, par):
    return ROOT.Math.crystalball_cdf(x[0], par[0], par[1], par[2], par[3])
#     tf_cb.SetParameters(par[0], par[1], par[2], par[3])
#     npts = 1000
#     x_ = []
#     w_ = []
   
#     return tf_cb.IntegralFast(npts, np.array(npts), np.array(npts), 0., x[0])
    #return tf_cb.Integral(0., x[0])
    
    
tf_cbi = ROOT.TF1('cbi', f_cbi, 0, 100, 4)
tf_cbi.SetParameters(10., 4., 3, 20.)
tf_cbi.SetLineColor(4)

c = newCanvas()
tf_cbi.Draw()
c.Draw()


    # ROOT::Math::crystalball_function (double x, double alpha, double n, double sigma, double x0=0)

# fitf_cb = ROOT.TF1('cbfunc', 'ROOT.TMath.crystalball_function (x, [0], [1], [2], [3])', 0, 100)
# 

# fitf_cb.SetParameters(1, 1, 1, 0)


In [ ]:
# Double_t turnon_func(Double_t *x, Double_t *par)
# {
#   double halfpoint = par[0];
#   double slope = par[1];
#   double plateau = par[2];

#   //double offset = par[3];
#   //double plateau = 1.0;
#   double offset = 0;

#   double pt = TMath::Max(x[0],0.000001);

#    double arg = 0;
#    //cout << pt <<", "<< halfpoint <<", " << slope <<endl;
#    arg = (pt - halfpoint)/(TMath::Sqrt(pt)*slope);
#    double fitval = offset+0.5*plateau*(1+TMath::Erf(arg));
#    return fitval;
# }




def turnon_func1(x, par):

    return 0.5 * par[0] * (1+ROOT.TMath.Erf((x[0]-par[1])/(par[2]*ROOT.TMath.Sqrt(2))))




def turnon_func(x, par):
  
    halfpoint = par[0]
    slope = par[1]
    plateau = par[2]
    offset = 0
    pt = ROOT.TMath.Max(x[0],0.000001)
    arg = 0
    arg = (pt - halfpoint)/(ROOT.TMath.Sqrt(pt)*slope)
    fitval = offset+0.5*plateau*(1+ROOT.TMath.Erf(arg))
    return fitval

fitf_to = ROOT.TF1('myfunc', turnon_func1, 0, 100, 3)
#fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf_to.SetParameters(1,25,2)
fitf_to.SetLineColor(3)

def fermi(x, params):
    return params[3]*(params[0] - 1/(ROOT.TMath.Exp((x[0]-params[1])/(params[2]))+1))
  
fitf = ROOT.TF1('myfunc', fermi, 0, 100, 4)
fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf.SetParameters(1,25,2,1)



c = newCanvas()
fitf.Draw()
c.Draw()


In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'DEFCalib', 'DEFMerged']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:
points

In [ ]:
# print points

points_calib_pu0 = [point for point in points if ('Calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'Calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if (point[0]=='DEF' and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if (point[0]=='DEF' and 'PU200' in point[1])]

points_merged_pu0 = [point for point in points if (point[0]=='DEFMerged' and 'PU0' in point[1])]
points_merged_pu200 = [point for point in points if (point[0]=='DEFMerged' and 'PU200' in point[1])]

# print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

def drawScaling(points, label, text):
    graph =  ROOT.TGraph(len(points), array('d', [point[2] for point in points]), array('d',[point[3] for point in points]))
    drawGraphsSame([graph], labels=[label], min_y=0, max_y=50, options='A*', text=text)
    graph.Fit('pol1', 'S')
    
    ROOT.gStyle.SetOptFit(11111)
    return graph

g1 = drawScaling(points=points_nocalib_pu0, label='PU0', text='PU0: no calib')
g3 = drawScaling(points=points_calib_pu0, label='PU0', text='PU0: calib')
g5 = drawScaling(points=points_merged_pu0, label='PU0', text='PU0: merged')

g2 = drawScaling(points=points_nocalib_pu200, label='PU200', text='PU200: no calib')
g4 = drawScaling(points=points_calib_pu200, label='PU200', text='PU200: calib')
g6 = drawScaling(points=points_merged_pu200, label='PU200', text='PU200: merged')


# graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
# drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
# graph_nocalib_pu0.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

# graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
# drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
# graph_calib_pu0.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)


# graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
# drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
# graph_nocalib_pu200.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

# graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
# drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
# graph_calib_pu200.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

In [ ]:
points = []


for pu in ['PU0', 'PU200']:
    for tp in ['HMvDR', 'HMvDRfixed']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:
from array import array
points_calib_pu0 = [point for point in points if ('fixed' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'fixed' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('fixed' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'fixed' not in point[0] and 'PU200' in point[1])]

print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
draw([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
draw([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
draw([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
draw([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

In [ ]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleAAA', 'eleAA', 'eleBB', 'eleCC', 'eleDD']


labels = [sample.label for sample in samples]




histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



In [ ]:
titles_df

In [ ]:
dir(histo_df.ele['DEF'][0])


In [ ]:
histo_df.loc['DEF', 'ele']

In [ ]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



In [ ]:
line095 = ROOT.TLine(0,0.95,110,0.95)
line095.SetLineStyle(2)
for particle in particles:
    for tp_set in tp_sets:
        #print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)
        line095.Draw('same')

In [ ]:
histo_den_df

In [ ]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

In [ ]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [ ]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



In [ ]:
dir(histo_df.photon['DEF'][0])


In [ ]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)
